# Settima lezione. Parte B

Importeremo i file da una repositoria su github.

In [12]:
baseURL    = 'https://raw.githubusercontent.com/'
user       = 'domenicozambella/'
repository = 'BioTeIndu19/'
branch     = 'master/'

## Ripasso lezione precedente

La seguente tabella contiene le velocità di reazioni misurate con diverse concentrazioni dell'inibitore: `v` è la velocità, `s` è la concentrazione del substrato, ed `i` è la concentrazione dell'inibitore.

In [13]:
import pandas as pd
file       = 'dati/mm3.csv'
df = pd.read_csv( baseURL + user + repository + branch + file )

Per fare la linearizzazione di Lineweaver-Burk calcoliamo i reciproci e li aggiungiamo come colonne al dataframe.

In [14]:
df['1/s'] = 1 / df['s']
df['1/v'] = 1 / df['v']

### Esplorazione grafica

Prima di fare la regressione lineare facciamo un'esplorazione grafica.

In [15]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()
param = dict(width = 700, height = 350,
             tools = 'wheel_zoom, reset,pan, box_zoom, crosshair',
            )

Loading BokehJS ...

Per separare i dati in 5 gruppi (uno per ogni valore di `i`) usiamo il metodo `groupby()`.

Ricordiamo che l'oggetto `gby` contiene coppie: 

    (concentrazione inibitore, dataframe corrispondente)

In [16]:
gby = df.groupby( 'i' )

In [17]:
from numpy import array    
from scipy.stats import linregress

p1 = figure(**param )
for i, sdf in gby:
    m, b, _, _, _ = linregress( sdf['1/s'], sdf['1/v'] )
    xmax = sdf['1/s'].max()     
    x = array( [-2, xmax] )
    y = m * x + b
    p1.line( x, y ) 
show( p1 )

## Migliorare la grafica

Proviamo a differenziare le varie curve con dei colori.

## Regressione non lineare

I dati suggeriscono che si tratti di un inibitore misto.

La velocità di reazione $v$ dipende dalla concentrazione di substrato $s$ e dell'inibitore $i$ e da 4 parametri $V_\textrm{max}$, $K_\textrm{m}$, $\alpha$, $\beta$ secondo la sequente equazione (copiata acriticamente da internet)

$$
v\ =\ \dfrac{V_\textrm{max}s}{K_\textrm{m} \big(1+i\cdot\alpha \big)+s \big(1+i\cdot\beta \big) }
$$

Con un procedimento di interpolazione non lineare si stimino i valori di questi 3 parametri.

Trascrivo in Python la funzione qui sopra (per leggibilità non uso la notazione `lambda`)

In [18]:
from scipy.optimize import curve_fit
def mmim( x, Vmax, Km, alpha, beta):
    s,i = x
    numeratore   = Vmax * s
    denumeratore = Km * (1 + i*alpha) + s * ( 1 + i*beta )
    return numeratore / denumeratore

Si faccia attenzione che il primo valore della funzione è letto come coppia di due valori. Questo è necessario perché `curve_fit()` vuole un'unica variabile per i dati indipendenti.

Fine

<hr><hr><hr>

La seguente cella importa file di stile HTML (può anche essere ignorata)

In [19]:
import requests
file = 'lezioni/style/custom.css'
from IPython.core.display import HTML
html_style = requests.get( baseURL + user + repository + branch + file ).text
HTML( html_style )